<a href="https://colab.research.google.com/github/zunyong01/hello-world/blob/main/lec27_notebook_for_students/swcon425_lec27_notebook_for_students.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lec 27 어텐션 메커니즘 구현하기

실습에 사용될 패키지를 미리 다운로드해주세요.

In [1]:
from importlib.metadata import version

print("파이토치 버전:", version("torch"))

파이토치 버전: 2.9.0+cu126


이번 실습 시간에는 LLM의 핵심 방법론인 어텐션 메커니즘을 다루겠습니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/01.webp?123" width="1000px">

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/02.webp" width="1000px">

## 3.1 긴 시퀀스 모델링의 문제점

트랜스포머(Transformer)는 원래 기계 번역 성능을 개선하기 위해 고안된 모델입니다.  
트랜스포머의 작동 원리를 알아보기 전에, 먼저 번역 task의 특성과 기존 방법론의 한계에 대해 알아보도록 하겠습니다.

번역을 수행할 때는 소스 언어와 타깃 언어 사이의 문법 구조 차이를 고려해야 합니다.  
서로 다른 언어 간의 문법 차이 때문에 텍스트는 한 단어씩 번역될 수 없습니다.

<img src="https://github.com/dusdnKR/SWCON425/blob/main/lec27_notebook_for_students/image/lec27_00.png?raw=1" width="1000px">

트랜스포머 모델이 소개되기 전에는 Seq2Seq 구조의 인코더-디코더 RNN이 기계 번역 작업에 널리 사용되었습니다.  

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/04.webp" width="1000px">

<b>인코더(Encoder)</b>는 소스 언어의 토큰 시퀀스를 순차적으로 처리하여 최종적으로 하나의 컨텍스트 벡터를 반환합니다.  
<b>디코더(Decoder)</b>는 압축된 정보를 담고 있는 컨텍스트 벡터를 토대로 타겟 언어의 토큰를 하나씩 생성합니다.

그림에서 볼 수 있듯, seq2seq 모델은 입력 토큰을 순서대로 처리하여 하나의 벡터를 반환합니다.  
때문에 문장이 길어지면 길어질수록 초반에 사용되었던 단어의 의미가 희미해지는 문제가 존재했고, 입력 문장의 길이에 관계없이 모든 정보를 고정된 벡터에 담아내야 한다는 점과, 이전 연산이 진행되어야 다음 연산 결과를 구할 수 있어 병렬 처리가 어렵다는 점이 한계로 작용했습니다.

**어텐션 메커니즘**은 이러한 Seq2Seq의 한계를 극복하기 위해 등장했습니다.

## 3.2 어텐션 메커니즘으로 데이터 종속성 포착하기

어텐션 메커니즘의 텍스트 생성 디코더도 모든 입력 토큰과 상호작용할 수 있습니다.  
seq2seq 모델과 다른 점은 생성하려는 단어와 관계성이 짙은 단어에 더 집중(attention)할 수 있다는 것입니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/05.webp" width="1000px">

그럼 어떤 단어에 더 attention해야 하는지 어떻게 알 수 있을까요?

## 3.3 셀프 어텐션으로 입력의 서로 다른 부분에 주의를 기울이기

### 3.3.1 훈련 가능한 가중치가 없는 간단한 셀프 어텐션 메커니즘

셀프 어텐션(self attention)이란, attention 방법론을 동일한 input에 대해 적용하는 것입니다.  
이해를 위해 <b>훈련 가능한 가중치가 없는</b> 간단한 버전의 셀프 어텐션을 구현하겠습니다.

-  $x^{(1)}$부터 $x^{(T)}$까지 하나의 입력 시퀀스가 주어졌다고 가정해 보겠습니다.
    - 입력은 텍스트(예를 들어 "Your journey starts with one step"와 같은 문장)이며, x는 각 토큰의 토큰 임베딩입니다.
    - 예를 들어 $x^{(1)}$은 단어 "Your"를 나타내는 d차원 벡터입니다.
- $x^{(1)}$에서 $x^{(T)}$ 사이에 있는 각 입력 시퀀스 원소 $x^{(i)}$에 대한 문맥 벡터(context vector) $z^{(i)}$를 계산하겠습니다. ($z$와 $x$는 차원이 같습니다)
    - 문맥 벡터는 어떠한 토큰을 기준으로 하는 전체 문맥 정보를 담고 있는 벡터입니다.
    - 문맥 벡터 $z^{(i)}$는 입력 $x^{(1)}$에서 $x^{(T)}$까지에 대한 가중치 합입니다.
    - 각각의 x에 곱해지는 가중치는 <b>어텐션 가중치(attention weight)</b>로, x의 기여도를 조절합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/07.webp" width="1000px">

구체적으로, 두 번째 입력 $x^{(2)}$에 대한 $z^{(2)}$를 구한다고 가정해보겠습니다.

우선 $x^{(2)}$과 $x^{(1)}$부터 $x^{(T)}$ 간의 <b>attention score</b>를 구해야 합니다.  
Attention score는 각 토큰 임베딩 벡터 간의 내적을 통해 계산됩니다. 이를 통해 벡터 간의 유사도를 수치화할 수 있습니다.

두 번째 입력 토큰을 쿼리로 사용한다면(즉, $q^{(2)} = x^{(2)}$), attention score $\omega$는 다음과 같이 계산됩니다.
- $\omega_{21} = x^{(1)} x^{(2)\top} = x^{(1)} q^{(2)\top}$
- $\omega_{22} = x^{(2)} x^{(2)\top} = x^{(2)} q^{(2)\top}$
- $\omega_{23} = x^{(3)} x^{(2)\top} = x^{(3)} q^{(2)\top}$
- ...
- $\omega_{2T} = x^{(T)} x^{(2)\top} = x^{(T)} q^{(2)\top}$

$\omega_{21}$에 있는 아래첨자 "21"은 입력 시퀀스 원소 1에 대해 입력 시퀀스 원소 2를 쿼리로 사용한다는 의미입니다.

3차원 벡터로 임베딩한 아주 간단한 입력 시퀀스가 있다고 가정해 보도록 하겠습니다.  
각 행은 단어를 나타내며, 각 열은 임베딩 차원을 나타냅니다.

In [2]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

# 3차원 임베딩 벡터라고 생각

두 번째 입력 원소 $x^{(2)}$를 쿼리로 사용해 context vector $z^{(2)}$를 계산해 보도록 하겠습니다.  
$x^{(2)}$와 다른 모든 입력 원소 사이의 dot product를 통해 attention score $\omega$를 계산합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/08.webp" width="1000px">

내적은 두 벡터 사이의 모든 원소를 곱하고 모두 합하여 구합니다.  
$x^{(2)}$와 $x^{(0)}$ 간의 dot product를 구해보겠습니다.

<img src="https://github.com/dusdnKR/SWCON425/blob/main/lec27_notebook_for_students/image/lec27_01.png?raw=1" width="500px">

In [ ]:
query = ### your code
input = ### your code
res = 0.

for idx, element in enumerate(input):
    res += ### your code

print(res)

tensor(0.9544)


`torch`의 `dot()` 함수를 이용하여 쉽게 계산할 수 있습니다.

In [ ]:
print(torch.dot(input, query))

tensor(0.9544)


$x^{(2)}$와 다른 모든 벡터 간의 dot product를 수행하여 attention score을 구해보도록 하겠습니다.

In [ ]:
query = ### your code

attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = ### your code

print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


다음으로, attention score ($\omega$)를 합이 1이 되도록 정규화하여 <b>attention weight</b>를 구해줍니다.  
정규화되지 않은 어텐션 점수를 합이 1이 되도록 정규화하는 간단한 방법은 모든 어텐션 점수의 합으로 나누는 것입니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/09.webp" width="1000px">

In [ ]:
attn_weights_2_tmp = ### your code

print("어텐션 가중치:", attn_weights_2_tmp)
print("합:", attn_weights_2_tmp.sum())

어텐션 가중치: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
합: tensor(1.0000)


실제로는 소프트맥스 함수로 정규화를 수행합니다. 극한 값을 잘 다루고 훈련 과정에 유용한 그레이디언트 속성을 제공하기 때문입니다.  
단순한 소프트맥스 함수를 이용하는 정규화 방식도 구현해보도록 하겠습니다.

<img src="https://github.com/dusdnKR/SWCON425/blob/main/lec27_notebook_for_students/image/lec27_02.png?raw=1" width="800px">

In [ ]:
def softmax_naive(x):
    return ### your code

attn_weights_2_naive = softmax_naive(attn_scores_2)

print("어텐션 가중치:", attn_weights_2_naive)
print("합:", attn_weights_2_naive.sum())

어텐션 가중치: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
합: tensor(1.)


이론적으로는 위와 같이 계산이 가능하지만, 최적화가 되지 않은 상태라 입력이 너무 크거나 작으면 오버플로우나 언더플로우가 발생할 수 있습니다.  
실제로는 최적화된 `torch` 라이브러리의 `softmax()` 함수를 사용하는 것이 좋습니다.

In [ ]:
attn_weights_2 = ### your code

print("어텐션 가중치:", attn_weights_2)
print("합:", attn_weights_2.sum())

어텐션 가중치: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
합: tensor(1.)


다음으로, 각각의 임베딩 토큰 $x^{(i)}$에 계산된 attention weignt를 곱하고 모두 합해 context vector $z^{(2)}$를 구합니다.  
쿼리 토큰 $x^{(2)}$에 대한 attention weignt 값이 높을수록 context vector에 더 큰 영향을 미치게 됩니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/10.webp" width="1000px">

In [ ]:
query = inputs[1]

context_vec_2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vec_2 += ### your code

print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


### 3.3.2 모든 입력 토큰에 대해 어텐션 가중치 계산하기

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/11.webp" width="1000px">

#### 모든 입력 토큰에 일반화하기

방금까지 self attention에 대해 알아보기 위하여 $x^{(2)}$에 대한  
<b>(1) attention score를 계산하고,  
(2) 합이 1이 되도록 정규화하여 attention weignt를 구한 뒤에,  
(3) 이 결과를 가중합하여 context vector를 만들었습니다.</b>

이 계산을 일반화하여 모든 attention weight와 context vector를 계산하겠습니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/12.webp" width="1000px">

우선, 모든 원소 쌍에 대한 attention score을 계산해보겠습니다.

In [ ]:
attn_scores = ### your code

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = ### your code

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


행렬 곱을 이용하면 동일한 행렬을 더 효율적으로 계산할 수 있습니다.

In [ ]:
attn_scores = ### your code
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


이제, 각 행의 값이 1이 되도록 정규화하여 attention weight를 계산합니다.

In [ ]:
attn_weights = ### your code
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


각 행의 합이 정말로 1이 되는지 확인해 보겠습니다.

In [ ]:
print(attn_weights.sum(dim=-1))

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


계산된 attention weight를 입력 임베딩에 곱하여 모든 context vector를 계산합니다.

In [ ]:
all_context_vecs = ### your code
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


이전에 계산했던 문맥 벡터 $z^{(2)}$와도 같은 결과인 것을 확인할 수 있습니다.

In [ ]:
print("이전에 계산한 두 번째 문맥 벡터:", context_vec_2)
print("방금 계산한 두 번째 문맥 벡터:", all_context_vecs[1])

이전에 계산한 두 번째 문맥 벡터: tensor([0.4419, 0.6515, 0.5683])
방금 계산한 두 번째 문맥 벡터: tensor([0.4419, 0.6515, 0.5683])


## 3.4 훈련 가능한 가중치를 가진 셀프 어텐션 구현하기

### 3.4.1 단계별로 어텐션 가중치 계산하기

원본 트랜스포머, GPT 모델 그외 다른 LLM에서 사용하는 셀프 어텐션 메커니즘을 구현해 보겠습니다.  
방금 구현한 기본적인 어텐션 메커니즘과의 차이는 <b>가중치 행렬 $W_q$, $W_k$, $W_v$</b>가 추가되었다는 것입니다.  
이 가중치 행렬은 학습 과정에서 업데이트되며 좋은 context vector를 만드는 방법을 학습하게 됩니다.

<img src="https://github.com/dusdnKR/SWCON425/blob/main/lec27_notebook_for_students/image/lec27_03.png?raw=1" width="1000px">

이전 과정과의 차이는 그림과 같습니다.  
추가된 가중치 행렬 $W_q$, $W_k$, $W_v$는 임베딩된 입력 토큰 $x^{(i)}$과 곱해져 각각 query, key, value 벡터로 사용됩니다.
  - $q^{(i)} = x^{(i)} \,W_q$
  - $k^{(i)} = x^{(i)} \,W_k$
  - $v^{(i)} = x^{(i)} \,W_v$

query, key, value라는 이름은 DB 용어에서 파생되었으며, 이름과 유사하게 각각 연산 기준, 비교 대상, 실제 벡터의 값의 역할을 수행합니다.  
그림을 통해 기존 방식과 비교했을 때 어떤 위치에서 사용되는지 알 수 있습니다.

입력 $x$와 쿼리 벡터 $q$의 임베딩 차원은 같은 수도 있고 다를 수도 있습니다. 모델 설계와 특정 구현에 따라 결정됩니다.  
GPT 모델에서는 입력과 출력 차원이 일반적으로 같습니다. 하지만 계산 과정을 설명하기 쉽도록 입력과 출력 차원을 다르게 하겠습니다.

In [ ]:
x_2 = inputs[1]
d_in = inputs.shape[1] # 입력 임베딩 크기, d=3
d_out = 2 # 출력 임베딩 크기, d=2

아래에서 세 개의 가중치 행렬을 초기화합니다.  
추후 `requires_grad=True`로 지정해서 훈련 과정 중에 이 행렬이 업데이트되도록 수정해야 합니다.

In [ ]:
torch.manual_seed(123)

W_query = ### your code
W_key   = ### your code
W_value = ### your code

각각의 가중치 행렬을 이용해 query, key, value를 계산해 보겠습니다.

In [ ]:
query_2 = ### your code
key_2 = ### your code
value_2 = ### your code

print(query_2)

tensor([0.4306, 1.4551])


이제 모든 입력 토큰에 대한 query, key, value를 구해보겠습니다.  
행렬의 크기를 확인해보면, 6개의 입력 토큰에 대한 3차원의 context vector가 2차원 임베딩 공간에 투영된 것을 확인할 수 있습니다.

In [ ]:
querys = ### your code
keys = ### your code
values = ### your code

print("inputs.shape:", inputs.shape)
print("querys.shape:", querys.shape)
print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

inputs.shape: torch.Size([6, 3])
querys.shape: torch.Size([6, 2])
keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


다음으로, 기준이 되는 query와 각각의 key 벡터 사이의 dot product를 수행해 attention score을 계산합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/15.webp" width="1000px">

In [ ]:
keys_2 = keys[1]
attn_score_22 = ### your code
print(attn_score_22)

tensor(1.8524)


이제 6개의 여섯 개의 입력 토큰 모두에 대해서도 구해보겠습니다.  
query 벡터에 대한 여섯 개의 attention score가 만들어집니다.

In [ ]:
attn_scores_2 = ### your code
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/16.webp" width="800px">

이제 소프트맥스 함수를 사용해 attention score를 정규화하여 attention weight를 구해보겠습니다.  
이전과 차이점은 소프트맥스 적용 전에 임베딩 차원의 제곱근 $\sqrt{d_k}$로 나누어 어텐션 점수의 스케일을 조정하는 것입니다.

<img src="https://github.com/dusdnKR/SWCON425/blob/main/lec27_notebook_for_students/image/lec27_02.png?raw=1" width="500px">

입력 값이 너무 커지거나 작아질 경우 기울기가 과도하게 작아지는 Gradient Vanishing 문제가 발생하여 학습에 악영향을 끼칠 수 있습니다.  
attention score을 $\sqrt{d_k}$로 나누는 것은 이러한 문제를 예방하기 위함입니다.

In [ ]:
d_k = keys.shape[1]
attn_weights_2 = ### your code
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/17.webp" width="1000px">

이제 두 번째 입력 query 벡터에 대한 context vector를 계산합니다.  
방금 구한 attention weight를 value 벡터에 곱하여 구합니다.

In [ ]:
context_vec_2 = ### your code
print(context_vec_2)

tensor([0.3061, 0.8210])


### 3.4.2 셀프 어텐션 파이썬 클래스 구현하기

이제 이 모든 과정을 한 번에 수행하는 `SelfAttention_v1`을 구현해 보겠습니다.

In [ ]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):

    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = ### your code
        self.W_key   = ### your code
        self.W_value = ### your code

    def forward(self, x):
        keys = ### your code
        queries = ### your code
        values = ### your code

        attn_scores = ### your code
        attn_weights = ### your code

        context_vec = ### your code
        return context_vec

torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/18.webp" width="1000px">

이 모든 과정은 파이토치의 `Linear` layer을 통해 더 간단하고 효율적으로 구현할 수 있습니다.  
방금 구현에 이용한 `nn.Parameter(torch.rand(...)`와 달리 `nn.Linear`는 가중치 초기화를 제공하여 더 안정적으로 모델을 훈련할 수 있습니다.

In [ ]:
class SelfAttention_v2(nn.Module):

    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = ### your code
        self.W_key   = ### your code
        self.W_value = ### your code

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


가중치 행렬 초깃값이 다르기 때문에 `SelfAttention_v1`와 `SelfAttention_v2`의 출력이 다릅니다.

## 3.5 코잘 어텐션으로 미래의 단어를 감추기

방금 구현한 셀프 어텐션 class는 행렬 연산을 통해 모든 연산을 빠르게 수행할 수 있도록 만들어졌습니다.  
하지만, 위와 같은 구조로 학습하게 되면 모델은 아직 알지 못하는 뒤의 토큰까지 attention 연산에 활용하게 됩니다.

<img src="https://github.com/dusdnKR/SWCON425/blob/main/lec27_notebook_for_students/image/lec27_04.png?raw=1" width="1000px">

코잘 어텐션(causal attention)은 주대각선 위의 attention weight를 마스킹합니다.  
이를 통해 행렬 연산으로 모든 계산을 한 번에 수행하면서도 새로운 context vector가 미래 토큰을 참조할 수 없도록 합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/19.webp" width="1000px">

### 3.5.1 코잘 어텐션 마스크 적용하기

코잘 어텐션을 실제로 구현해보도록 하겠습니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/20.webp" width="1000px">

방금 만든 `SelfAttention_v2` 객체의 쿼리와 키 가중치 행렬을 기반으로 구현해보도록 하겠습니다.

In [ ]:
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)
attn_scores = queries @ keys.T

attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


미래 시점의 attention weight를 마스킹하는 가장 간단한 방법은 파이토치 `tril` 함수를 이용하는 것입니다.  
이를 통해 주대각선과 그 아래의 원소는 1, 주대각선 위의 원소는 0인 마스크를 만들 수 있습니다.

In [ ]:
context_length = attn_scores.shape[0]
mask_simple = ### your code
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


그다음 attention weight와 마스크를 곱해서 주대각선 위의 값을 0으로 만듭니다.

In [ ]:
masked_simple = ### your code
print(masked_simple)

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


하지만 마스크를 소프트맥스 함수 이후에 적용하면 정규화 확률 분포가 망가지게 됩니다.  

In [ ]:
print(masked_simple.sum(dim=-1))

tensor([0.1921, 0.3700, 0.5357, 0.6775, 0.8415, 1.0000],
       grad_fn=<SumBackward1>)


각 행의 합이 1이 되도록 attention weight를 다시 정규화할 수 있습니다.  
마스킹된 각 행의 합을 구해 나누어 줍니다.

In [ ]:
masked_simple_norm = ### your code
print(masked_simple_norm)
print(masked_simple_norm.sum(dim=-1))

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)
tensor([1., 1., 1., 1., 1., 1.], grad_fn=<SumBackward1>)


이론적으로는 코잘 어텐션 메커니즘의 구현이 완료되었습니다.  
하지만, 정규화를 두 번 수행하는 것보다 더 효율적인 방법이 필요하겠죠.

<img src="https://github.com/dusdnKR/SWCON425/blob/main/lec27_notebook_for_students/image/lec27_02.png?raw=1" width="500px">

소프트맥스 함수의 특성을 이용하면 정규화를 한 번만 수행하고도 동일한 결과를 얻을 수 있습니다.  
소프트맥스는 음의 무한대가 입력으로 들어오면 출력이 0에 가까워집니다.

마스킹될 토큰의 attention score을 음의 무한대(`-inf`)로 대치시키겠습니다.

In [ ]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = ### your code
print(masked)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


이 attention score에 소프트맥스 정규화를 적용하면 마스킹된 attention weight을 얻을 수 있습니다.

In [ ]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


### 3.5.2 드롭아웃으로 어텐션 가중치에 추가적으로 마스킹하기

모델 훈련에서는 overfitting을 막기 위해 종종 드롭아웃(dropout) 기법을 사용합니다.  
이 방법은 랜덤한 값을 추가로 마스킹하여 모델이 훈련 데이터에 과도하게 의지하지 못하도록 막습니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/22.webp" width="600px">

드롭아웃 마스크를 구현해보도록 하겠습니다.  
본 실습에서는 마스크의 효과를 편하게 확인하게 위해 드롭아웃 비율 $p$를 50%으로 지정하였습니다.  
하지만 추후 학습에는 0.1이나 0.2 정도의 비율을 사용하는 것이 좋습니다.

In [ ]:
torch.manual_seed(123)
dropout = ### your code
example = torch.ones(6, 6)

print(dropout(example))

tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])


드롭아웃 마스크 적용 결과를 확인해보면, 단순히 마스킹만 적용되는 것이 아니라 원본 값에도 변화가 생긴 것을 확인할 수 있습니다.  
드롭아웃 마스크는 사라진 값을 보완하기 위해 남아있는 값에 $\frac{1}{p}$를 곱해줍니다. 현재는 $p=0.5$이므로, $\frac{1}{0.5}=2$배가 적용되었습니다.  
실제 `attn_weights`에도 적용해보겠습니다.

In [ ]:
torch.manual_seed(123)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.8966, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4921, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4350, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)


### 3.5.3 코잘 어텐션 클래스 구현하기

이제 코잘 마스크와 드롭아웃 마스크가 적용된 셀프 어텐션 클래스를 만들어보겠습니다.  
또한, 지난 실습에서 만든 데이터 로더의 배치 출력도 지원할 수 있도록 배치 처리를 고려해서 구현하겠습니다.

batch size가 2이고, 각각의 batch가 6개의 토큰으로 구성된 3차원 임베딩 벡터가 있다고 가정하겠습니다.

In [ ]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape)

torch.Size([2, 6, 3])


코잘 마스크와 드롭아웃 마스크가 적용된 셀프 어텐션 클래스를 구현합니다.

In [ ]:
class CausalAttention(nn.Module):

    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        # self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = ### your code
        ### your code

    def forward(self, x):
        ### your code
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2)
        ### your code
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = ### your code

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)

context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)

context_vecs = ca(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


## 3.6 싱글 헤드 어텐션을 멀티 헤드 어텐션으로 확장하기

### 3.6.1 여러 개의 싱글 헤드 어텐션 층 쌓기

이러한 어텐션 모듈 각각을 헤드(head)라고 부릅니다.  
방금 구현한 `CausalAttention` 클래스는 한 번의 어텐션 과정을 수행하는 <b>싱글 헤드 어텐션</b>과 같습니다.

<img src="https://github.com/dusdnKR/SWCON425/blob/main/lec27_notebook_for_students/image/lec27_05.png?raw=1" width="800px">

실제 트랜스포머 모델에서는 여러 개의 헤드를 가진 <b>멀티 헤드 어텐션</b>을 수행합니다.  
각각의 헤드는 학습 가능한 고유한 가중치 행렬 $W_q$, $W_k$, $W_v$를 가지고 있습니다.  
즉, 각각의 헤드는 서로 다른 방향으로 학습되어 다양한 관점에서 attention을 수행하게 됩니다.

<img src="https://github.com/dusdnKR/SWCON425/blob/main/lec27_notebook_for_students/image/lec27_08.png?raw=1" width="800px">

예를 들어, 어떤 방향으로 학습이 이루어졌는지에 따라 문장의 종류에 집중하거나, 특정 품사에 집중하거나, 품사 간의 관계에 집중할 수 있을 것입니다.

<img src="https://github.com/dusdnKR/SWCON425/blob/main/lec27_notebook_for_students/image/lec27_06.png?raw=1" width="800px">

위 이미지는 실제 어텐션 논문에 실린 attention 시각화 자료입니다.  
"its"라는 단어를 기준으로 attention weight가 높은 단어를 확인해 보았을 때, 보라색 head는 "its"라는 대명사의 본래 의미에 집중하고 있으며, 갈색 head는 본래 의미를 나타내는 "Law"와 "its"가 꾸며주는 "application"에 동시에 집중하고 있습니다.

<img src="https://github.com/dusdnKR/SWCON425/blob/main/lec27_notebook_for_students/image/lec27_07.png?raw=1" width="800px">

멀티 헤드 어텐션의 결과는 단순히 각 헤드의 결과를 concat한 것과 같습니다.  
그러므로, 이전에 구현한 `CausalAttention`을 여러 개 쌓아서 결과를 합치는 것만으로도 멀티 헤드 어텐션을 구현할 수 있습니다.

In [ ]:
class MultiHeadAttentionWrapper(nn.Module):

    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [] ### your code
        )

    def forward(self, x):
        return ### your code


torch.manual_seed(123)

context_length = batch.shape[1] # 토큰 개수
d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(
    d_in, d_out, context_length, 0.0, num_heads=2
)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


위 구현에서 임베딩 차원은 4입니다. 문맥 벡터는 물론 쿼리, 키, 값 벡터의 차원으로 `d_out=2`를 지정했기 때문입니다.  
두 개의 어텐션 헤드가 있으므로 출력 임베딩 차원은 2*2=4가 됩니다.

### 3.6.2 가중치 분할로 멀티 헤드 어텐션 구현하기

위에서 구현한 내용을 더 효율적으로 만들어 보겠습니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/26.webp" width="800px">

`CausalAttention` 여러 개를 각각 계산하여 concat하는 대신, 하나의 행렬이 여러 헤드 구역으로 나뉘어 있다고 생각하고 한 번의 행렬 연산으로 계산을 수행할 수 있도록 구현하겠습니다.

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out은 num_heads로 나누어 떨어져야 합니다"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = ### your code

        self.W_query = ### your code
        self.W_key = ### your code
        self.W_value = ### your code
        self.out_proj = nn.Linear(d_out, d_out) # Linear 층을 사용해 헤드의 출력을 결합합니다 (최근 연구에서는 이를 제거해도 성능에 영향을 미치지 않는다고 함)
        self.dropout = ### your code
        ### your code

    def forward(self, x):
        b, num_tokens, d_in = x.shape # b: 배치 차원, num_tokens: 토큰 개수, d_in: 입력 임베딩 크기

        keys = self.W_key(x) # 크기: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)
        print("key.shape before reshape:", keys.shape)
        print("b:", b, ", num_tokens:", num_tokens, ", d_out:", self.d_out)
        print()

        # `num_heads` 차원을 추가함으로써 암묵적으로 행렬을 분할합니다.
        # 그다음 마지막 차원을 `num_heads`에 맞춰 채웁니다: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = ### your code
        values = ### your code
        queries = ### your code
        print("keys.shape after reshape:", keys.shape)
        print("d_out:", self.d_out, "= num_heads:", self.num_heads, "* head_dim:", self.head_dim)
        print()

        # 전치: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = ### your code
        queries = ### your code
        values = ### your code
        print("keys.shape after transpose:", keys.shape)
        print("b:", b, ", num_heads:", self.num_heads, ", num_tokens:", num_tokens, ", head_dim:", self.head_dim)
        print()

        # 코잘 마스크로 스케일드 점곱 어텐션(셀프 어텐션)을 계산합니다.
        attn_scores = ### your code
        print("attn_scores.shape:", attn_scores.shape)
        print("[6,2] @ [6,2]T = [6,6]")
        print()

        # 마스크를 불리언 타입으로 만들고 토큰 개수로 마스크를 자릅니다.
        mask_bool = ### your code

        # 마스크를 사용해 어텐션 점수를 채웁니다.
        ### your code

        attn_weights = ### your code
        attn_weights = ### your code

        # 크기: (b, num_tokens, num_heads, head_dim)
        context_vec = ### your code
        print("context_vec.shape after transpose:", context_vec.shape)
        print("b:", b, ", num_tokens:", num_tokens, ", num_heads:", self.num_heads, ", head_dim:", self.head_dim)
        print()

        # 헤드를 결합합니다. self.d_out = self.num_heads * self.head_dim
        context_vec = ### your code
        context_vec = self.out_proj(context_vec) # 투영
        print("context_vec.shape after reshape:", context_vec.shape)
        print("b:" , b, ", num_tokens:", num_tokens, ", d_out:", self.d_out)
        print()

        return context_vec

torch.manual_seed(123)

batch_size, context_length, d_in = batch.shape
d_out = 4
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

key.shape before reshape: torch.Size([2, 6, 4])
b: 2 , num_tokens: 6 , d_out: 4

keys.shape after reshape: torch.Size([2, 6, 2, 2])
d_out: 4 = num_heads: 2 * head_dim: 2

keys.shape after transpose: torch.Size([2, 2, 6, 2])
b: 2 , num_heads: 2 , num_tokens: 6 , head_dim: 2

attn_scores.shape: torch.Size([2, 2, 6, 6])
[6,2] @ [6,2]T = [6,6]

context_vec.shape after transpose: torch.Size([2, 6, 2, 2])
b: 2 , num_tokens: 6 , num_heads: 2 , head_dim: 2

context_vec.shape after reshape: torch.Size([2, 6, 4])
b: 2 , num_tokens: 6 , d_out: 4

tensor([[[ 0.1184,  0.3120, -0.0847, -0.5774],
         [ 0.0178,  0.3221, -0.0763, -0.4225],
         [-0.0147,  0.3259, -0.0734, -0.3721],
         [-0.0116,  0.3138, -0.0708, -0.3624],
         [-0.0117,  0.2973, -0.0698, -0.3543],
         [-0.0132,  0.2990, -0.0689, -0.3490]],

        [[ 0.1184,  0.3120, -0.0847, -0.5774],
         [ 0.0178,  0.3221, -0.0763, -0.4225],
         [-0.0147,  0.3259, -0.0734, -0.3721],
         [-0.0116,  0.3138, -0.07

`MultiHeadAttention`는 `MultiHeadAttentionWrapper`를 더 효율적으로 재작성한 것입니다.  
각 단계의 shape을 출력 결과를 통해 확인할 수 있습니다.

`MultiHeadAttention`와 `MultiHeadAttentionWrapper`클래스는 모두 `d_out=2`를 사용하지만, 약간 다릅니다.  
`MultiHeadAttentionWrapper`는 헤드의 출력을 연결하기 때문에 출력 차원이 `d_out * num_heads`입니다(즉, `2*2 = 4`).  
하지만 `MultiHeadAttention` 클래스는 `num_heads`와 무관하게 출력 차원이 `d_out`입니다.

파이토치에서는 방금 구현한 멀티 헤드 어텐션 함수를 지원합니다.  
더 효율적인 구현을 위해 [`torch.nn.MultiheadAttention`](https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html) 클래스를 사용할 수 있습니다.

<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
본 강의자료는 세바스찬 라시카(Sebastian Raschka)의 <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> (<a href="<a href="http://tensorflow.blog/llm-from-scratch">밑바닥부터 만들면서 배우는 LLM</a>)의 예제를 참고하여 제작되었습니다.
</font>
</td>
</tr>
</table>